# Soduku solver
Little fun project for auto solving soduku puzzles

In [1]:
import numpy as np

soduku = np.arange(81).reshape(9,9)

In [2]:
# Helper functions for debugging
def fill_soduku_with_block_ids(game):
    for x in range(3):
        for y in range(3):
            game[0 + y * 3, 0 + x * 3] = -(x + y * 3)
            game[0 + y * 3, 1 + x * 3] = -(x + y * 3)
            game[0 + y * 3, 2 + x * 3] = -(x + y * 3)
            game[1 + y * 3, 0 + x * 3] = -(x + y * 3)
            game[1 + y * 3, 1 + x * 3] = -(x + y * 3)
            game[1 + y * 3, 2 + x * 3] = -(x + y * 3)
            game[2 + y * 3, 0 + x * 3] = -(x + y * 3)
            game[2 + y * 3, 1 + x * 3] = -(x + y * 3)
            game[2 + y * 3, 2 + x * 3] = -(x + y * 3)

# Some live hack functions
# block{x, y} is the block position (start with zero)
# cell{x, y} is the position within the cell
def fill_cell(game, block_id, cell_y, cell_x, val):
    x = (block_id % 3);
    y = round((block_id - x) / 3)

    x *= 3
    y *= 3

    game[y + cell_y, x + cell_x] = val


Fill below the current game.  
Each line of code is used to fill 1 cell.  

Example: soduku[3, 4] = 5  
This will fill a cell with the value 5 at the location y 3 and x 4  
The left top corner is x=0 and y=0  

You can also use the helper function "fill_cell"

The blocks are aranged in the following manner:  
0 1 2  
3 4 5  
6 7 8  

In [3]:
# Clear old game
soduku.fill(0)

## Example

# block 0
fill_cell(soduku, 0, 0, 0, 6)
fill_cell(soduku, 0, 1, 0, 2)
fill_cell(soduku, 0, 0, 2, 5)
fill_cell(soduku, 0, 1, 2, 7)

# block 1
fill_cell(soduku, 1, 0, 2, 2)
fill_cell(soduku, 1, 2, 1, 9)

# block 2
fill_cell(soduku, 2, 0, 1, 1)
fill_cell(soduku, 2, 1, 1, 6)
fill_cell(soduku, 2, 1, 2, 5)

# block 3
fill_cell(soduku, 3, 0, 0, 7)
fill_cell(soduku, 3, 0, 2, 4)

# block 4
fill_cell(soduku, 4, 1, 0, 3)

# block 5
fill_cell(soduku, 5, 0, 1, 8)
fill_cell(soduku, 5, 1, 2, 7)
fill_cell(soduku, 5, 2, 0, 6)
fill_cell(soduku, 5, 2, 1, 4)
fill_cell(soduku, 5, 2, 2, 1)

# block 6
fill_cell(soduku, 6, 1, 0, 8)
fill_cell(soduku, 6, 1, 2, 9)

# block 7
fill_cell(soduku, 7, 0, 1, 8)
fill_cell(soduku, 7, 1, 2, 1)
fill_cell(soduku, 7, 2, 0, 2)
fill_cell(soduku, 7, 2, 1, 5)

# block 8
fill_cell(soduku, 8, 0, 1, 9)
fill_cell(soduku, 8, 1, 0, 4)
fill_cell(soduku, 8, 1, 1, 2)
fill_cell(soduku, 8, 1, 2, 6)

## Fill your values (don't forget to comment the example)
# soduku[y, x] = val OR fill_cell(soduku, ...)


In [310]:
print(soduku)

[[6 0 5 0 0 2 0 1 0]
 [2 0 7 0 0 0 0 6 0]
 [0 0 0 0 9 0 0 0 0]
 [7 0 4 0 0 0 0 8 0]
 [0 0 0 3 0 0 0 0 7]
 [0 0 0 0 0 0 6 4 1]
 [0 0 0 0 8 0 0 9 0]
 [8 0 9 0 0 1 4 2 6]
 [0 0 0 2 0 0 0 0 0]]


First create the option matrix and setup some important functions

In [6]:
transform = np.array([1, 2, 4, 8, 16, 32, 64, 128, 256])

init_value = transform.sum()

def has_single_bit(val):
    return val and not(val & (val - 1))

import math

def get_upperleft_corner(y, x):
    return math.floor(y / 3) * 3, math.floor(x / 3) * 3

def propagate_cell_change(game, y, x, val, options): 
    c_y, c_x = get_upperleft_corner(y, x)
    
    options[y, x] = 0
    flag = transform[val - 1]

    # Reduce block
    block_update_mask = ((options[c_y:c_y+3,c_x:c_x+3] & flag) != 0).astype(int)
    block_update_mask *= flag

    options[c_y:c_y+3,c_x:c_x+3] -= block_update_mask

    # Reduce line
    line_update_mask = ((options[y] & flag) != 0).astype(int)
    line_update_mask *= flag

    options[y] -= line_update_mask

    # Reduce stripe
    stripe_update_mask = ((options[:, x] & flag) != 0).astype(int)
    stripe_update_mask *= flag

    options[:, x] -= stripe_update_mask 

def set_cell_val(game, y, x, val, options):
    game[y, x] = val
    propagate_cell_change(game, y, x, val, options)

def solve_cell(game, y, x, options):
    cell_options = options[y,x]

    if has_single_bit(cell_options):
        set_cell_val(game, y, x, math.floor(math.log2(cell_options) + 1), options)
        return True
    else:
        active_flags = []
        for i in range(8, -1, -1):
            if (cell_options & transform[i]) != 0:
                active_flags += [(i, transform[i])]

        for active_flag in active_flags:
            index, flag = active_flag

            c_y, c_x = get_upperleft_corner(y, x)

            c = ((options[c_y:c_y+3,c_x:c_x+3] & flag) != 0).astype(int).sum() # Blocks
            if c == 1:
                set_cell_val(game, y, x, index + 1, options)
                return True
            else:
                c = ((options[y] & flag) != 0).astype(int).sum() # Lines
                if c == 1:
                    set_cell_val(game, y, x, index + 1, options)
                    return True
                else:
                    c = ((options[:, x] & flag) != 0).astype(int).sum() # Stripes
                    if c == 1:
                        set_cell_val(game, y, x, index + 1, options)
                        return True    
    return False

It's time to solve the soduku puzzle

In [7]:
# Let clear options setup
options_matrix = np.arange(81).reshape(9,9)
options_matrix.fill(init_value)

# Make a backup
soduku_backup = soduku.copy()

# No available options for preset values
for y in range(9):
    for x in range(9):
        if soduku[y,x] > 0:
            propagate_cell_change(soduku, y, x, soduku[y,x], options_matrix)


print('start\n', soduku)

old_sum = options_matrix.sum()
new_sum = -1

c = 0

while old_sum != new_sum:
    c += 1
    
    for y in range(9):
        for x in range(9):
            if options_matrix[y,x] > 0:
                solve_cell(soduku, y, x, options_matrix) 
            
    old_sum = new_sum
    new_sum = options_matrix.sum()                           

print('solution\n', soduku)
print('cycles required ', c)

# Enable for debugging
# print(options_matrix)

# Restore backup
soduku = soduku_backup

start
 [[6 0 5 0 0 2 0 1 0]
 [2 0 7 0 0 0 0 6 5]
 [0 0 0 0 9 0 0 0 0]
 [7 0 4 0 0 0 0 8 0]
 [0 0 0 3 0 0 0 0 7]
 [0 0 0 0 0 0 6 4 1]
 [0 0 0 0 8 0 0 9 0]
 [8 0 9 0 0 1 4 2 6]
 [0 0 0 2 5 0 0 0 0]]
solution
 [[6 3 5 8 7 2 9 1 4]
 [2 9 7 4 1 3 8 6 5]
 [4 1 8 5 9 6 7 3 2]
 [7 2 4 1 6 5 3 8 9]
 [9 6 1 3 4 8 2 5 7]
 [5 8 3 9 2 7 6 4 1]
 [1 7 2 6 8 4 5 9 3]
 [8 5 9 7 3 1 4 2 6]
 [3 4 6 2 5 9 1 7 8]]
cycles required  8
